In [1]:
from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction
from fireworks.core.rocket_launcher import rapidfire
from fw_tutorials.dynamic_wf.fibadd_task import FibonacciAdderTask

In [2]:
mod_name = __name__
def fibonacci_adder(num1, num2, stop_point=0):
    if num1 + num2 < stop_point:
        new_fw = Firework(
            PyTask({
                'func': mod_name + '.fibonacci_adder', 
                'args': [num2, num1 + num2, stop_point]
            }))
        print(f"The current number is {num1+num2}")
        return FWAction(additions=new_fw)
    else:
        print(f"The limit was reached, the next number is {num1+num2}")

In [3]:
# set up the LaunchPad and reset it
loglvl = 'CRITICAL'
launchpad = LaunchPad(strm_lvl=loglvl)
launchpad.reset('', require_password=False)

In [4]:
# No appending FWs
firework = Firework(
    PyTask({
        "func": mod_name + '.fibonacci_adder', 
        "args":[0, 1, 0]
    }))
launchpad.add_wf(firework)
rapidfire(launchpad, FWorker(), strm_lvl=loglvl)

2018-11-13 10:35:23,610 INFO RUNNING fw_id: 1 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-23-604200
2018-11-13 10:35:23,614 INFO Task started: PyTask.
The limit was reached, the next number is 1
2018-11-13 10:35:23,615 INFO Task completed: PyTask 
2018-11-13 10:35:23,621 INFO Rocket finished


In [5]:
# No appending FWs
firework = Firework(
    PyTask({
        "func": mod_name + '.fibonacci_adder', 
        "args":[0, 1, 1000]
    }))
launchpad.add_wf(firework)
rapidfire(launchpad, FWorker(), strm_lvl=loglvl)

2018-11-13 10:35:23,795 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-23-794929
2018-11-13 10:35:23,796 INFO Launching Rocket
2018-11-13 10:35:23,816 INFO RUNNING fw_id: 2 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-23-794929
2018-11-13 10:35:23,824 INFO Task started: PyTask.
The current number is 1
2018-11-13 10:35:23,825 INFO Task completed: PyTask 
2018-11-13 10:35:23,837 INFO Rocket finished
2018-11-13 10:35:23,840 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-23-840354
2018-11-13 10:35:23,840 INFO Launching Rocket
2018-11-13 10:35:23,850 INFO RUNNING fw_id: 3 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-23-840354
2018-11-13 10:35:23,854 INFO Task started: PyTask.
The current number is 2
2018-11-13 10:35:23,854 INFO Task completed: PyTask 
2018-

The limit was reached, the next number is 1597
2018-11-13 10:35:24,237 INFO Task completed: PyTask 
2018-11-13 10:35:24,245 INFO Rocket finished


In [6]:
# Import modules for ase calculator inputs
import os
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db.core import connect

from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction, Workflow
from fireworks.core.rocket_launcher import launch_rocket, rapidfire

# Minimal hilde inputs
from hilde.structure.structure import pAtoms
from hilde.helpers.converters import dict2patoms, patoms2dict
from hilde.helpers.hash import hash_atoms

mod_name = __name__

In [7]:
def calculate(atoms_dict):
    at = dict2patoms(atoms_dict)
    at.calc.calculate(at, properties=['forces'])
    return FWAction(update_spec={"calc_atoms": patoms2dict(at)})

In [8]:
def calc_to_db(db_path, atoms_dict):
    db = connect(db_path)
    at = dict2patoms(atoms_dict)
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), ("calc_hash", "=", calc_hash)]
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [9]:
# Initialize Structure
db_path = (os.getcwd() + '/test.db')
print(f'database: {db_path}')

ni = pAtoms(bulk('Ni', cubic=True))
ni.set_calculator(EMT())
ni_dict = patoms2dict(ni)

ni_hash, calc_hash = hash_atoms(ni)

database: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/test.db


In [10]:
ft_calc = PyTask({"func": mod_name+".calculate", "args": [ni_dict]})
ft_to_db = PyTask({"func":mod_name+".calc_to_db", "args":[db_path], "inputs":["calc_atoms"]})

In [11]:
# set up the LaunchPad and reset it
launchpad = LaunchPad(strm_lvl="CRITICAL")
launchpad.reset('', require_password=False)

launchpad.add_wf(Firework([ft_calc, ft_to_db]))
launch_rocket(launchpad, strm_lvl="CRITICAL")

2018-11-13 10:35:24,580 INFO Launching Rocket
2018-11-13 10:35:24,589 INFO RUNNING fw_id: 1 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial
2018-11-13 10:35:24,593 INFO Task started: PyTask.
2018-11-13 10:35:24,623 INFO Task completed: PyTask 
2018-11-13 10:35:24,626 INFO Task started: PyTask.
2018-11-13 10:35:24,677 INFO Task completed: PyTask 
2018-11-13 10:35:24,693 INFO Rocket finished


True

In [12]:
db = connect(db_path)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash)], columns=["forces"]))[0]
print(f"Atomic forces are:\n{row.get('forces')}")

Atomic forces are:
[[ 2.75670600e-15 -1.06217932e-16 -5.86336535e-16]
 [ 2.08575155e-14 -2.32452946e-15 -1.40512602e-15]
 [-2.75474088e-15  2.34534614e-15  1.25940924e-15]
 [-9.96193416e-15 -2.80117861e-15  2.01227923e-16]]


In [13]:
# Initialize Structure Al
al = pAtoms(bulk('Al', 'fcc', a=4.047266))
al.set_calculator(EMT())
al_dict = patoms2dict(al)

al_hash, calc_hash = hash_atoms(al)

In [14]:
fw_calc = Firework(PyTask({"func": mod_name+".calculate", "args": [al_dict]}))
fw_to_db = Firework(PyTask({"func":mod_name+".calc_to_db", "args":[db_path], "inputs":["calc_atoms"]}))
wf = Workflow([fw_calc, fw_to_db], {fw_calc:[fw_to_db]})
launchpad.add_wf(wf)
rapidfire(launchpad, nlaunches=2, strm_lvl="CRITICAL")

2018-11-13 10:35:24,752 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-752037
2018-11-13 10:35:24,753 INFO Launching Rocket
2018-11-13 10:35:24,773 INFO RUNNING fw_id: 3 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-752037
2018-11-13 10:35:24,778 INFO Task started: PyTask.
2018-11-13 10:35:24,805 INFO Task completed: PyTask 
2018-11-13 10:35:24,816 INFO Rocket finished
2018-11-13 10:35:24,819 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-819367
2018-11-13 10:35:24,820 INFO Launching Rocket
2018-11-13 10:35:24,828 INFO RUNNING fw_id: 2 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-819367
2018-11-13 10:35:24,832 INFO Task started: PyTask.
2018-11-13 10:35:24,862 INFO Task completed: PyTask 
2018-11-13 10:35:24,869 INFO Rocket finished


In [15]:
db = connect(db_path)
at = db.get_atoms(calc_hash=calc_hash, atoms_hash=al_hash, attach_calculator=True)
print(f"Atomic forces are, {at.get_forces()}")

Atomic forces are, [[0. 0. 0.]]


In [16]:
import numpy as np
from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms
from hilde.phonon_db.row import phonon2dict, PhononRow
from hilde.phonon_db.phonon_db import connect as connect_ph

def to_phonopy_atoms(atoms):
    phonopy_atoms = PhonopyAtoms(
        symbols=atoms.get_chemical_symbols(),
        cell=atoms.get_cell(),
        masses=atoms.get_masses(),
        positions=atoms.get_positions(wrap=True),
    )
    return phonopy_atoms

def calculate(atoms_dict):
    at = dict2patoms(atoms_dict)
    at.calc.calculate(at, properties=['forces'])
    return FWAction(mod_spec=[{'_push': {"calc_forces": patoms2dict(at)}}])

def calculate_multiple(atom_dicts):
    firework_detours = []
    for i, cell in enumerate(atom_dicts):
        task = PyTask({"func": mod_name+".calculate",
                       "args": [cell]})
        firework_detours.append(Firework(task, name=f"calc_{i}"))
    return FWAction(detours=firework_detours)

def get_fcs(phonon_dict, atoms_ideal, calc_atoms):
    atoms = dict2patoms(atoms_ideal)
    disp_cells = [dict2patoms(ca) for ca in calc_atoms]
    disp_cells = sorted(disp_cells, key=lambda x: x.info['disp_num'])
    phonon = PhononRow(phonon_dict).to_phonon()
    phonon.generate_displacements(distance=0.01)
    phonon.set_forces([cell.get_forces() for cell in disp_cells])
    phonon.produce_force_constants()
    return FWAction(update_spec={"phonon_dict": phonon2dict(phonon)})

def calc_to_db(db_path, sc_dict, phonon_dict):
    db = connect_ph(db_path)
    at = dict2patoms(sc_dict)
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), 
                 ("calc_hash", "=", calc_hash), 
                 ("supercell_matrix", "=", phonon_dict["supercell_matrix"])]
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, phonon_dict, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(phonon_dict, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [17]:
smatrix = np.array([3, 0, 0, 0, 3, 0, 0, 0, 3]).reshape(3,3)
db_path = os.getcwd() + "test_ph.db"

ph_atoms = to_phonopy_atoms(ni)

phonon = Phonopy(ph_atoms, supercell_matrix=smatrix, symprec=1e-5, is_symmetry=True, factor=15.633302)
phonon_dict = phonon2dict(phonon)
pc_dict = patoms2dict(ni)
phonon.generate_displacements(distance=0.01)
scs = phonon.get_supercells_with_displacements()
sc_dicts = []
for i, sc in enumerate(scs):
    scs[i] = pAtoms(phonopy_atoms=sc)
    scs[i].info['disp_num'] = i
    scs[i].calc = ni.calc
    sc_dicts.append(patoms2dict(scs[i]))

In [18]:
ft_initialize = PyTask({"func": mod_name + ".calculate_multiple", "args":[sc_dicts]})
ft_get_fc = PyTask({"func": mod_name + ".get_fcs", "args":[phonon_dict, pc_dict], "inputs":["calc_forces"]})
ft_calc_to_db = PyTask({"func": mod_name + ".calc_to_db", "args":[db_path, pc_dict], "inputs":["phonon_dict"]})

In [19]:
fw_initialize = Firework(ft_initialize)
fw_get_fc = Firework(ft_get_fc)
fw_calc_to_db = Firework(ft_calc_to_db)

wf = Workflow([fw_initialize, fw_get_fc, fw_calc_to_db], {fw_initialize:[fw_get_fc], fw_get_fc: [fw_calc_to_db]})
launchpad.add_wf(wf)
rapidfire(launchpad, nlaunches=0, strm_lvl="CRITICAL")

2018-11-13 10:35:25,324 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-324373
2018-11-13 10:35:25,325 INFO Launching Rocket
2018-11-13 10:35:25,335 INFO RUNNING fw_id: 6 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-324373
2018-11-13 10:35:25,340 INFO Task started: PyTask.
2018-11-13 10:35:25,341 INFO Task completed: PyTask 
2018-11-13 10:35:25,360 INFO Rocket finished
2018-11-13 10:35:25,364 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-364235
2018-11-13 10:35:25,364 INFO Launching Rocket
2018-11-13 10:35:25,374 INFO RUNNING fw_id: 7 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-364235
2018-11-13 10:35:25,380 INFO Task started: PyTask.
2018-11-13 10:35:25,544 INFO Task completed: PyTask 
2018-11-13 10:35:25,569 INFO Rocket finished
2018-11-

In [20]:
db = connect_ph(db_path)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash), 
                                ("supercell_matrix", "=", list(smatrix.flatten()))], 
                     columns=["force_constants"]))[0]
print(f"The force constants are:\n{row.get('force_constants')}")

The force constants are:
[[[[ 1.14463612e+01 -0.00000000e+00 -0.00000000e+00]
   [-0.00000000e+00  1.14463612e+01 -0.00000000e+00]
   [-0.00000000e+00 -0.00000000e+00  1.14463612e+01]]

  [[ 2.37234745e-01 -1.26217745e-29  2.52435490e-29]
   [-0.00000000e+00 -2.88493428e-02 -0.00000000e+00]
   [-0.00000000e+00  2.52435490e-29 -2.88493428e-02]]

  [[ 2.37234745e-01 -0.00000000e+00  2.52435490e-29]
   [-0.00000000e+00 -2.88493428e-02 -0.00000000e+00]
   [-0.00000000e+00 -6.31088724e-30 -2.88493428e-02]]

  ...

  [[ 4.47709709e-03 -5.98183255e-03 -1.50460276e-02]
   [-5.98183255e-03  4.47709709e-03  1.50460276e-02]
   [-1.50472523e-02  1.50472523e-02  2.56372784e-02]]

  [[-5.51410327e-05 -0.00000000e+00 -0.00000000e+00]
   [-0.00000000e+00  9.90223749e-08 -1.47542017e-05]
   [-0.00000000e+00 -1.47530139e-05 -2.43796562e-05]]

  [[ 4.47709709e-03  5.98183255e-03  1.50460276e-02]
   [ 5.98183255e-03  4.47709709e-03  1.50460276e-02]
   [ 1.50472523e-02  1.50472523e-02  2.56372784e-02]]]


